In [21]:
import minsearch
import json

In [22]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [23]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [26]:
index.fit(documents)

In [27]:
q = 'the course has already started, can I still enroll?'

In [28]:
boost = {'question': 3.0, 'section': 0.5} # Peso das seções


results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5
)

In [29]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [30]:
boost = {'question': 3.0, 'section': 0.5} # Peso das seções
filter = {'course': 'data-engineering-zoomcamp'}

results_de = index.search( # Filtra apenas fad de data engineering
    query=q,
    filter_dict=filter,
    boost_dict=boost,
    num_results=5
)

results_de

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Passando  o contexto pro LLM

In [31]:
from openai import OpenAI

In [32]:
client = OpenAI(api_key="sk-proj-xxx")

In [33]:
response = client.chat.completions.create(
                model='gpt-3.5-turbo', # 'gpt-4o' pago
                messages=[{"role": "user", "content": q}]
            ) # Resposta genérica (Erro porque não coloquei o cartão ainda pra billar)



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
prompt_template = """

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.

Use only the facts from the CONTEXT when answering the QUESTION.

If the CONTEXT doesn't contain the answer, output NONE.


QUESTION: {question}

CONTEXT:
{context}



""" # Remove espaço

In [34]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']} \n question: {doc['question']} \n answer: {doc['text']} \n\n"


In [35]:
print(context)

section: General course-related questions 
 question: The course has already started. Can I still join it? 
 answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate. 

section: General course-related questions 
 question: Course - Can I still join the course after the start date? 
 answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

section: General course-related questions 
 question: Course - Can I follow the course after it finishes? 
 answer: Yes, we will keep all the materials after the course fini

In [36]:
prompt = prompt_template.format(question=q, context=context).strip()

In [37]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.

Use only the facts from the CONTEXT when answering the QUESTION.

If the CONTEXT doesn't contain the answer, output NONE.


QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions 
 question: The course has already started. Can I still join it? 
 answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate. 

section: General course-related questions 
 question: Course - Can I still join the course after the start date? 
 answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there 

In [38]:
response_with_context = client.chat.completions.create(
                model='gpt-3.5-turbo', # 'gpt-4o' pago
                messages=[{"role": "user", "content": prompt}]
            ) # Resposta genérica (Erro porque não coloquei o cartão ainda pra billar)



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}